In [2]:
# Importing Dependencies
from Pipeline_Support.ETL_MasterFunction import etl_master
from Pipeline_Support.DimensionalQueries import dimquery
from Pipeline_Support.FactSnapshot import create_fact_snapshot
from Pipeline_Support.FTSUpload import upload_fact_table
from IPython.display import Image

## Populating Database
First we need to generate synthetic data from Mockaroo for our defined database schema and upload the datasets on a GitHub repo which we'll be using as a proxy for a database

In [ ]:
%run Pipeline_Support/DataGen.py

### ETL
The following operations are done in the ETL Pipeline:
* It fetches the datasets (csv tables) from the GitHub Repo and PostgreSQL and stores all of them as dataframes in a dictionary.
* Then it treats for the missing values in each of the dataframes: some of the features like rent_amount, sale_amount, commission_value, commission_rate is filled logically, other numeric features are imputed through KNN imputer (each dataframe separately), and finally categorical features are filled with modal value.
* Then it corrects the data types for various columns in each dataframe where needed.
* Then the star schema is created as per the blueprint already defined:
    * First Date dimension is created which has dates generated from 01/01/2022 to 31/12/2025 with a unique data key.
    * Then Location dimension is generated and location id from the oltp table is kept along with a unique location.
    * Similarly, Agent, Property Details, and Listing dimensions are generated.
    * Then the fact table is generated by doing multiple joins (left dataframe merges) to get a big dataframe with all the necessary features required to create facts and then once facts are created unnecessary columns are dropped. Finally, fact dataframe is merged with each dimensional dataframe based on the oltp ids kept in the dimensions to get dimension foreign keys in the fact table. Lastly, oltp ids are dropped from dimensions.
    

In [3]:
import sys
sys.path.append(r"D:\Github\DWM-Project")

Dim_Date, Dim_Location, Dim_Agent, Dim_PropertyDetails, Dim_Listing, Fact_Transaction = etl_master(
    source="hybrid",
    db_params={
        'user': 'postgres',
        'password': 'asifa123',
        'host': 'localhost',
        'port': 5432,
        'db_name': 'Real-Estate-Management'
    }
)


🔄 Fetching from both CSV + PostgreSQL...
✅ Data ingestion complete.


d:\Github\DWM-Project\E2E_DWH_Pipeline\Pipeline_Support\ETL_SupportFunctions.py:214: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_value, inplace=True)
d:\Github\DWM-Project\E2E_DWH_Pipeline\Pipeline_Support\ETL_SupportFunctions.py:214: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

✅ Missing values filled.
🧹 Removed 1 duplicate rows from 'contract'
✅ Data types corrected.
✅ Star schema generated successfully.
✅ All CSVs saved successfully in E2E_DWH_Pipeline folders.
⬆️ Loading tables into PostgreSQL database...
✅ Data successfully loaded into PostgreSQL!


### Potential Analytical Requirements of the Business
**The following analytical queries are answered through our created data mart (star schema). For further details, check DimensionalQueries.py:**

1- What was the total commission generated through agents handling rental listings in the first quarter of 2021?

2- What was the average time taken to close a listing by agents handling sale listings in all four quarters of 2022?

3- What was average commission rate of the top 5 cities with the most sale transactions in 2023?

4- What was the average time spent in negotiations by agents for the top 3 states with the most rental transactions in 2022 and 2023?

5- What was the average commission generated by middle-aged male and female agents in 2022?

6- What was the average transaction value for Broker Associate in the fourth quarter of 2023? 

7- What was average number of transactions handled by agents who've joined in the past two years?

8- What was the average time taken to close a sale listing where property size was greater 3000 sqft and had more than 4 bedrooms in 2021?

9- What is the average maintenance cost and demand price discount for properties which are built in last two decades?

10- What is the average time taken to close a listing which has a property in a poor condition?

In [4]:
dimquery(Dim_Date=Dim_Date,Dim_Location=Dim_Location,Dim_Agent=Dim_Agent,Dim_PropertyDetails=Dim_PropertyDetails,Dim_Listing=Dim_Listing,Fact_Transaction=Fact_Transaction)

Query 1
+---+--+
| 0 |  |
+---+--+ 


Query 2
+---+---------+
| 0 | 341.543 |
+---+---------+ 


Query 3
+---+---------------+---------+
| 0 | Oklahoma City | 1.51667 |
| 1 | Chicago       | 1.51667 |
| 2 | San Francisco | 2.125   |
| 3 | New York      | 2.125   |
| 4 | Los Angeles   | 1.615   |
+---+---------------+---------+ 


Query 4
 


Query 5
+---+--------+---------+
| 0 | Female | 14886.8 |
| 1 | Male   | 22059.5 |
+---+--------+---------+ 


Query 6
+---+--+
| 0 |  |
+---+--+ 


Query 7
+---+-----+
| 0 | 304 |
+---+-----+ 


Query 8
+---+--+
| 0 |  |
+---+--+ 


Query 9
+---+---------+---------+
| 0 | 4473.43 | 36.8899 |
+---+---------+---------+ 


Query 10
+---+---------+
| 0 | 679.125 |
+---+---------+ 




### Fact Table Snapshot
Fact table snapshot is created by simply joining the fact table with dimensional tables based on the dimensional keys and which gives in return a big dataframe that has all the dimensional attributes of all dimensional tables within the fact table (based on their respective keys which were foreign keys in the fact table) along with dimensional keys and finally dimensional keys are then dropped from the fact table.

Then the fact table snapshot is uploaded on the GitHub repo as a csv file.

In [7]:
Fact_Snap=create_fact_snapshot(Dim_Date=Dim_Date,Dim_Location=Dim_Location,Dim_Agent=Dim_Agent,Dim_PropertyDetails=Dim_PropertyDetails,Dim_Listing=Dim_Listing,Fact_Transaction=Fact_Transaction)

🧩 Checking Fact_Transaction date coverage...
   DateID
0     536
1     536
2     536
3     536
4     536
Unique DateIDs: 10
✅ Fact Snapshot saved successfully at:
D:\Github\DWM-Project\E2E_DWH_Pipeline\Fact Table Snapshot\Fact_Snapshot.csv
✅ Fact_Transaction table saved at:
D:\Github\DWM-Project\E2E_DWH_Pipeline\Fact Table Snapshot\Fact_Transaction.csv
✅ Both Fact Snapshot and Fact Transaction uploaded to PostgreSQL successfully!


In [ ]:
upload_fact_table(file_content=Fact_Snap.to_csv(index=False),file_name='FactSnapshot.csv')